#### <b> Lab 3 - Week 1 | Day 4 <b>

<b>Build a Web chatbot that acts like You, using Gradio and OpenAI</b>

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [ ]:
load_dotenv(override=True)
openai = OpenAI()

In [ ]:
# Read the LinkedIn PDF file using the pypdf library

reader = PdfReader("me/linkedin.pdf")
linkedin_text = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin_text += text

In [ ]:
print(linkedin_text)

In [ ]:
# Define the name of the person the chatbot is representing
# This is used to personalize the chatbot's responses
name = "Ed Donner"

# Read the summary of the person the chatbot is representing
# This is used to provide context to the chatbot's responses
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:
print(summary)

In [ ]:
# Define the system prompt for the chatbot which uses GPT-4o-mini model

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin_text}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [ ]:
print(system_prompt)

In [ ]:
# Define a chat function that Gradio can use

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message.content

In [ ]:
# Launch the chatbot

gr.ChatInterface(chat, type="messages", title="Ed Donner's Web Chatbot", description="Ask me anything!").launch()

<b>Use Gemini to evaluate the Chatbot (gpt-4o-mini) responses: A Multi LLM Pipeline</b>

1. Ask an LLM to evaluate an answer
2. Rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [ ]:
# Create a pydantic model for the evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
# Define the system prompt for the evaluator

evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin_text}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
# Define a function to create the user prompt for the evaluator

def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n {message} \n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt


In [ ]:
import os

# Create openRouter client for gemini. This follows the standard OpenAI format with a base URL to openrouter

gemini = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key= os.getenv('OPENROUTER_API_KEY'), # Replace with your actual key
)

In [ ]:
# Define the evaluate function which returns a "Structured output" (JSON) in the Schema defined using pydantic's BaseModel class

def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content":evaluator_system_prompt}] + [{"role": "user", "content":evaluator_user_prompt(reply, message, history)}]
    respose = gemini.beta.chat.completions.parse(model="google/gemini-2.5-flash", messages=messages, response_format=Evaluation)
    return respose.choices[0].message.parsed

In [ ]:
# Ask the Chatbot a question

messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
print(reply)

In [ ]:
# Now, call the evaluate function with the reply received from the LLM, tell the question and pass the history 
# messages[:1] - represents first element(list slicing) which is the system_prompt
# This will in turn invoke gemini and ask it to build an evaluator object representing its feedback about the answer

evaluate(reply, "do you hold a patent?", messages[:1])

In [ ]:
# Define a rerun function
# If the chatbot's original answer is rejected by the evaluator, the feedback is passed on to the chatboat to rerun

def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"

    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
# Modified chat function

def chat(message, history):

    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
                it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed Evaluation - returning reply")
    else:
        print("Failed Evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)

    return reply
        



In [ ]:
gr.ChatInterface(chat, type="messages").launch()